In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot



In [3]:
df_colcap = pd.read_csv('../data/raw/colcap_to_cons.csv')
df_colcap['Fecha'] = pd.to_datetime(df_colcap['Fecha'])
df_colcap = df_colcap.set_index('Fecha')

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/colcap_to_cons.csv'

In [ ]:
df_colcap.describe()

For an initial analysis it is possible to observe some basic statistical data that give a general context of
the information with which it is useful to work


## Plotting Price Trend and Returns

Based on the economic context of the data, the variables that are of greatest interest to analyze
the performance of Colcap indicator value are the closing price because it shows the final daily valuation
and the returns that allow us to visualize the rate of change of action from day to day.


In [ ]:
ax1 = df_colcap.plot(use_index=True, y='Último', style=['-'], figsize=(25,10), ylabel='Último', legend=False, grid=True)
ax2 = df_colcap.plot(use_index=True, y='% var.', style=['-'], figsize=(25,10), ylabel='Variacion%', legend=False, grid=True)


plt.title('COLCAP')
;

A great performance can be observed between the years 2009 - 2011, followed by a lateralization period of approximately four years (until 2015). After this, a fall to approximately 10,200 to generate a recovery to over 16,000 in 2020. However, as in many economies, the impact of the covid 19 health emergency significantly affected the value, falling to approximately 10,000. In recent years it has been recovering as the emergency has been resolved.
Regarding the variation, there are two outstanding periods. In the year 2009 where there was a significant loss prior to the great upward momentum that took the value of the indicator to record highs. And the other period to highlight is the effect of covid 19 that generated the greatest historical variation

## Dotted Plot

Given that the dataset has a daily variation and we have a dataset that covers a period of 14 years,
a dotted graph allows us to see in a little more detail the gap between one sample and the other
for both the closing price and the price variation

In [ ]:
plt.figure()
ax3 = df_colcap['Último'].plot(style='k.', grid=True, figsize=(25,10), ylabel='Closing Price')
plt.figure()
ax4 = df_colcap['% var.'].plot(style='k.', grid=True, figsize=(25,10), ylabel='Variacion%')



;

## Histogram of Prices and Returns

Another valuable analysis is knowing the distribution of our variables. One of the most common
ways is through a histogram that evaluates and graphically represents the frequency of the data.
The price for its variation does not present a normal distribution. However, from these data it can
be deduced that the control point (higher frequency) is approximately 13,000 COP, but this one does not look very solid, since there are frequencies almost at the same level in the prices 14,000 - 17,000.
On the other hand, the variation presents an apparently normal distribution with some very extreme
points reaching below -10% and almost 5%

In [ ]:
plt.figure()
ax5 = df_colcap['Último'].hist(bins=30, legend=True)

plt.figure()
ax6 = df_colcap['% var.'].hist(bins=50, legend=True)

;

## Kernel Density Plot

Continuing with the previous analysis, a better way to understand the behavior of the price and
the variation is by analyzing its density, which by smoothing some data gives us an idea of the
prices or variations where the confluence has been greater

In [ ]:
plt.figure()
ax6 = df_colcap['Último'].plot(kind='kde', grid=True, legend=True)
plt.figure()
ax6 = df_colcap['% var.'].plot(kind='kde', grid=True, legend=True)

;

## Plotting Moving Average

Another useful strategy to analyze time series is to reduce the variation of the data by applying
filters. A first approximation that we present is through a simple moving average comprised of a
window of 200 periods that presents the average evolution of the price based on the previous 200
data. A window of 200 is used because it is an industry standard measure and is used by many
institutions to assess the value of stocks.

In [ ]:
ax1 = df_colcap.plot(use_index=True, y='Último', style=['-'], figsize=(25,10), ylabel='Closing Price', legend=False, grid=True)
price_mean = df_colcap['Último'].rolling(200, win_type=None).mean()
price_mean.plot(style=['-', '--', ':'], grid=True, ylabel='Closing Price')

ax22 = df_colcap.plot(use_index=True, y='% var.', style=['-'], figsize=(25,10), ylabel='Variation %', legend=False, grid=True)
price_mean = df_colcap['% var.'].rolling(200, win_type=None).mean()
price_mean.plot(style=['-', '--', ':'], grid=True, ylabel='Variation %')

;

## Exponential Smoothing

Since the ultimate goal is to make a prediction model, it is important to reduce the variation of the
variable by smoothing it, however, a linear alternative such as the moving average usually generates
time shifts in the data, so non-linear options such as the exponential smoothing allows noise to be
smoothed without time shift implications


In [ ]:
model = SimpleExpSmoothing(df_colcap['Último']).fit(smoothing_level=.1, optimized=True)

df_colcap['Último'].plot(marker='o', color='red', ylabel='Closing Price')
model.fittedvalues.plot(marker='.', color='blue')

;


## Lag Plot 

It is used to check if the series is random or not. Lag plot allows to do a price analysis between the observation of the sample in *t* and *t+1* where it is tried to verify if there is any pattern that shows that the series is not random, in other words to verify the existence of correlation between the samples.

 In the case of the closing price of the Colcap indicator, there is a positive correlation, which finally implies that its behavior can be modeled.
 A structure can then be observed that invites us to think of a linear trend in the evolution of the price.

In [ ]:
lag_plot(df_colcap['Último'])

;

## Autocorrelation Plot

Autocorrelation plots are a commonly-used tool for checking randomness in a data set. This
randomness is ascertained by computing autocorrelations for data values at varying time lags. If
random, such autocorrelations should be near zero for any and all time-lag separations. If nonrandom, then one or more of the autocorrelations will be significantly non-zero.
Reinforcing the previous conclusion, it can be seen that the correlation of the data is  far
from zero in several segments, so the variable of the closing price of the share does not have a random component that
predominates its evolution over time.


In [ ]:
autocorrelation_plot(df_colcap['Último'])

;

Most of the daily volume traded is 10,000 million COP or less, there are some days with higher volumes but not very frequent. This figure indicates a market with decent liquidity that may attract investors to consider this asset in portfolios.

## Yearly price and variation trend (seasonal plot)

In [ ]:
df_colcap.reset_index(inplace=True)
#df_colcap.insert(2, 'year',  df_colcap['fecha'].apply(lambda x: x.year))
#df_colcap.insert(3, 'month',  df_colcap['fecha'].apply(lambda x: x.month))
#df_colcap.insert(4, 'day',  df_colcap['fecha'].apply(lambda x: x.day))
z = pd.DataFrame(df_colcap.groupby(by = ['year', 'month'])['Último'].mean())
z.reset_index(inplace=True)
z.set_index('month', inplace=True)


In [ ]:
import numpy as np
for i in z['year'].unique():
    z[z['year'] == i]['Último'].plot(label=i)

    

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylabel('Closing Price')
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.xticks(np.arange(0, 13, 1))
plt.title('COLCAP - Average closing price per month')
plt.show()




There is no clear pattern in the average price trend. There are years where through the months the price has been a bit flat, has fallen or has appreciated a bit.

Even for months that can be compared taking into account seasonality, there is no consistent structure that can determine any forecast or index


In [ ]:
w = pd.DataFrame(df_colcap.groupby(by = ['year', 'month'])['% var.'].mean())
w.reset_index(inplace=True)
w.set_index('month', inplace=True)



In [ ]:
import numpy as np
for i in w['year'].unique():
    w[w['year'] == i]['% var.'].plot(label=i)

    

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylabel('Volume (millions)')
plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.xticks(np.arange(0, 13, 1))
plt.title('COLCAP - Variation per month')
plt.grid()
plt.show()

In general, the variation is in a range between 0.5% and -0.25%. However, there are data a bit far away with variations of -1.25%, -0.75% and almost 0.75%. So it is to be expected that the daily variation of the Colcap index is in the mentioned range, since the vast majority of days since 2008 have been in this range.

In [ ]:
x = z.copy()
x.reset_index(inplace=True)
x.set_index(["month", "year"], inplace=True)
x.sort_index(inplace=True)

for i in range(1, 13, 1):
    x[np.in1d(x.index.get_level_values(0), [i])].plot(label=i)
    plt.axhline(int(x[np.in1d(x.index.get_level_values(0), [i])].mean()),color="red")
    plt.grid()
    plt.ylabel('Closing Price')
    plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
    plt.title(i)


;

A pattern can be observed in the behavior of the share price in the respective
months of the different years. There is an increase in value
for each of the periods, and this is preceded by a
drop in the price. In general, the index shows consistency in growth,
red line is the average of the prices and an average
similar around 1,400 COP which is consistent with the price of the point
of control mentioned in the previous points

In [ ]:
df_colcap.head(2)

In [ ]:
df_out = pd.DataFrame()
df_out['Date'] = pd.to_datetime(df_colcap['fecha'])
df_out['Instrument Name'] = df_colcap['Nemotecnico']
df_out['Year'] = df_colcap['year']
df_out['Month'] = df_colcap['month']
df_out['Day'] = df_colcap['day']
df_out['Quantity'] = df_colcap['Cantidad']
df_out['High'] = df_colcap['Precio Mayor']
df_out['Mid'] = df_colcap['Precio Medio']
df_out['Low'] = df_colcap['Precio Menor']
df_out['Close'] = df_colcap['Precio Cierre']
df_out['Close previous'] = df_colcap['Precio Cierre'].shift(1)
df_out['Volume'] = df_colcap['Volumen']
df_out['Variation'] = df_colcap['Precio Cierre'].pct_change()*100.0
df_out = df_out.set_index('Date').dropna()
df_out

In [ ]:
path_out = '../data/dev/ISA.csv'
df_out.to_csv(path_out, encoding='utf-8')